In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Загружаем файл routes_by_stop
rbs_df = pd.read_csv('resources/rbs.csv', ';')

In [ ]:
# Вспомогательная функция, которая по номеру маршрута, дате и типу транспорта выдает список остановок в указанную дату в двух направлениях
# Формат результата (4 параметра): количество остановок в прямом направлении, перечень остановок в прямом направлении, 
# количество остановок в обратном направлении, перечень остановок в обратном направлении

def get_route_stops(route_number, tr_date, tr_id):
    route_df = rbs_df[rbs_df['ROUTE_NUMBER'] == route_number]
    route_df = route_df[route_df['DDATE'] == tr_date]
    route_df = route_df[route_df['ID_TRANSPORT'] == tr_id]
    route_stops_in_dir1 = route_df[route_df['DIRECTION'] == 1].ID_STOP.values
    route_stops_in_dir2 = route_df[route_df['DIRECTION'] == 2].ID_STOP.values
    num1 = route_stops_in_dir1.shape[0]
    num2 = route_stops_in_dir2.shape[0]
    if route_stops_in_dir1.shape[0] != 0:
        dir1_str = str(route_stops_in_dir1[0])
    else: 
        dir1_str = ""
    if route_stops_in_dir2.shape[0] != 0:
        dir2_str = str(route_stops_in_dir2[0])
    else: 
        dir2_str = ""
    for stop in route_stops_in_dir1[1:]:
        dir1_str += ","
        dir1_str += str(stop)
    for stop in route_stops_in_dir2[1:]:
        dir2_str += ","
        dir2_str += str(stop)
    return num1, dir1_str, num2, dir2_str

In [ ]:
# Пример работы функции get_route_stops
route_number = '68'
tr_date = '03.10.2018'
tr_id = 1 # это значит, что мы хотим получить информацию об автобусе №68

num1, ex1, num2, ex2 = get_route_stops(route_number, tr_date, tr_id)

print('Количество остановок на городском маршруте №' + route_number + ' в день ' + tr_date + ' в направлении 1: ' + str(num1))
print('Количество остановок на городском маршруте №' + route_number + ' в день ' + tr_date + ' в направлении 2: ' + str(num2))

In [ ]:
#Исследуем файл rbs на предмет поиска дат, в которые было больше всего информации о маршрутах
max_val = 0
days = []
months = []

for i in ['01', '02','03','04','05','06','07','08','09', '10',
         '11', '12','13','14','15','16','17','18','19', '20',
         '21', '22','23','24','25','26','27','28','29', '30', '31']:
    for j in ['08', '09', '10']:
        cur_date = i + '.' + j + '.2018'
        num_routes = rbs_df[rbs_df['DDATE'] == cur_date]['ROUTE_NUMBER'].unique().shape[0] 
        if num_routes > max_val:
            max_val = num_routes
            
for i in ['01', '02','03','04','05','06','07','08','09', '10',
         '11', '12','13','14','15','16','17','18','19', '20',
         '21', '22','23','24','25','26','27','28','29', '30', '31']:
    for j in ['08', '09', '10']:
        cur_date = i + '.' + j + '.2018'
        num_routes = rbs_df[rbs_df['DDATE'] == cur_date]['ROUTE_NUMBER'].unique().shape[0] 
        if num_routes == max_val:
            days.append(cur_date.split('.')[0])
            months.append(cur_date.split('.')[1])
            
print("Информация о самом большом количестве маршрутов (" + str(max_val) + " маршрута) отражена в документе в следующие дни:")
for day, month in zip(days, months):
    print(day + '.' + month + '.2018')

In [ ]:
# Функция, которая позволяет создать единый csv-файл, в котором будет отражена 
# вся информация о маршрутах определенного типа (автобусы в т.ч. коммерческие, трамваи, троллейбусы) в указанный день 
# и остановках на них (в прямом и обратном направлении)

def create_routes_by_stops_csv(tr_date, tr_id):
    all_routes_df = rbs_df[rbs_df['DDATE'] == tr_date]
    all_routes = all_routes_df[all_routes_df['ID_TRANSPORT'] == tr_id]['ROUTE_NUMBER'].unique()
    result_data = []
    for i in range(all_routes.shape[0]):
        route_number = all_routes[i]
        dir1 = ""
        dir2 = ""
        num1, dir1, num2, dir2 = get_route_stops(route_number, tr_date, tr_id)
        result_data.append([str(route_number), tr_id, num1, dir1, num2, dir2])
    return result_data

In [ ]:
# Итоговый csv будет содержать 6 колонок: 
# route_number - номер маршрута, 
# id_transport - id вида транспорта, к которому относится маршрут (1 - автобус, в т.ч. коммерческие; 2 - трамвай; 3 - троллейбус),
# number_of_stops_in_direction1 - количество остановок в прямом направлении, 
# stops_in_direction1 - перечень ID остановок в прямом направлении через запятую в строковом формате, 
# number_of_stops_in_direction2 - количество остановок в обратном направлении, 
# stops_in_direction2 - перечень ID остановок в обратном направлении через запятую в строковом формате

# Выбираем дату (по умолчанию стоит дата, в которую больше всего записей о маршрутах)
tr_date = '22.09.2018'

# Готовим даннные для всех типов транспорта
bus_data = create_routes_by_stops_csv(tr_date, 1)
tram_data = create_routes_by_stops_csv(tr_date, 2)
troll_data = create_routes_by_stops_csv(tr_date, 3)

# Записываем итоговые dataset-ы в файлы (3 отдельных csv файла для автобусов, трамваев и троллебусов)
routes_by_stops_df = pd.DataFrame(np.array(bus_data), columns=['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2'])
routes_by_stops_df.to_csv('resources/buses_routes_by_stops.csv', encoding='utf-8-sig')

routes_by_stops_df = pd.DataFrame(np.array(tram_data), columns=['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2'])
routes_by_stops_df.to_csv('resources/tram_routes_by_stops.csv', encoding='utf-8-sig')

routes_by_stops_df = pd.DataFrame(np.array(troll_data), columns=['route_number', 'id_transport', 'number_of_stops_in_direction1', 'stops_in_direction1', 'number_of_stops_in_direction2', 'stops_in_direction2'])
routes_by_stops_df.to_csv('resources/troll_routes_by_stops.csv', encoding='utf-8-sig')

In [ ]:
# Открываем получившийся dataset
bus_df = pd.read_csv('resources/buses_routes_by_stops.csv', index_col=0)
tram_df = pd.read_csv('resources/tram_routes_by_stops.csv', index_col=0)
troll_df = pd.read_csv('resources/troll_routes_by_stops.csv', index_col=0)

# Для удобства также создадим общий dataset, в котором будет содержаться информация о всех видах транспорта
r_df = bus_df
r_df = r_df.append(tram_df, ignore_index = True)
r_df = r_df.append(troll_df, ignore_index = True)
r_df = r_df.fillna('-1')

# Блок корректировки данных о маршрутах (дальнейшая проверка выявила ошибки в исходных данных об остановках)
r_df.loc[r_df['route_number'] == '467', 'number_of_stops_in_direction2'] = 4
r_df.loc[r_df['route_number'] == '467', 'stops_in_direction2'] = '2530,4398,3006,4403'
r_df.loc[r_df['route_number'] == '467', 'number_of_stops_in_direction1'] = 3
r_df.loc[r_df['route_number'] == '467', 'stops_in_direction1'] = '18506,22763,4379'

r_df.loc[r_df['route_number'] == '469', 'number_of_stops_in_direction1'] = 13
r_df.loc[r_df['route_number'] == '469', 'stops_in_direction1'] = '18506,22763,4379,1310,2026,1313,1314,2529,20595,17650,17652,1319,1320'
r_df.loc[r_df['route_number'] == '469', 'number_of_stops_in_direction2'] = 14
r_df.loc[r_df['route_number'] == '469', 'stops_in_direction2'] = '22764,20596,2533,2532,17654,2530,22002,2528,2527,3212,2524,4398,3006,4403'

In [ ]:
# Ищем нужные нам коммерческие маршруты (проверка актуальности информации)
# Данные 11 коммерческих маршрутов - самые популярные среди пассажиров, оплачивающих проезд картами "Подорожник"
# Именно поэтому они представляют наибольший интерес

route_nums = ['К-68', 'К-191', 'К-92', 'К-49', 'К-55', 'К-13', 'К-7', 'К-67', 'К-184', 'К-1', 'К-226']

print("Результаты поиска данных о самых популярных коммерческих маршрутах в документе rbs.csv")
print()

bus_df = r_df[r_df['id_transport'] == 1]

for route_num in route_nums:
    print('----------------------------------------')
    print('Маршрут ', route_num)
    print('Количество остановок в прямом направлении: ', bus_df[bus_df['route_number'] == route_num]['number_of_stops_in_direction1'].values[0])
    print('Количество остановок в обратном направлении: ', bus_df[bus_df['route_number'] == route_num]['number_of_stops_in_direction2'].values[0])

Как мы видим, информация по остановкам для нужных нам коммерческих маршрутов (тех, по которым есть много информации о транзакциях по картам "Подорожник") некорректна (т.к. фактически на указанных маршрутах гораздо больше остановок).

Для корректной дальнешей работы был проведен дополнительный анализ указанных маршрутов и корректная информация о них описана ниже. Тем не менее, отдельный интерес представляют также те коммерческие маршруты, для которых информация по количеству остановок корректна. Поэтому следующий блок содержит код функции, позволяющей находить коммерческие маршруты, для которых описанное количество остановок дотаточно большое.

In [ ]:
# Вначале определим полный список коммерческих маршрутов, информация о которых есть в документе
commercial_routes = []
for route_num in r_df['route_number'].values:
    if str(route_num).split('-')[0] == 'К':
        commercial_routes.append(route_num)

# Далее создадим функцию, позволяющую определить топ 10 коммерческих маршрутов с самым большим количеством остановок
def find_most_informative_routes():
    number_of_stops_1 = []
    number_of_stops_2 = []
    for num in commercial_routes:
        number_of_stops_1.append(r_df[r_df['route_number'] == num]['number_of_stops_in_direction1'].values[0])
        number_of_stops_2.append(r_df[r_df['route_number'] == num]['number_of_stops_in_direction2'].values[0])
    zipped_info = zip(commercial_routes, number_of_stops_1, number_of_stops_2)
    new_zip = sorted(zipped_info, key = lambda x: x[1], reverse=True)
    return new_zip[:10]

In [ ]:
# Самые информативные маршруты
res = find_most_informative_routes()

In [ ]:
# Следующий блок кода посвящен добавлению в получившийся датасет с информацией об общественном транспорте информацию о кластерах
# (districts), которым принадлежат остановки на маршруте

stations_df = pd.read_csv('resources/stations.csv', ';')
clusters_df = pd.read_csv('resources/district_stops.csv', ';')

def find_cluster_id(id_stop):
    res = clusters_df[clusters_df['ID_STOP'] == int(float(id_stop))]['ID_STOP_GROUP'].values
    if res.shape[0] == 0:
        return '-1'
    else:
        return str(int(res[0]))

In [ ]:
# Проверка информации по похожим маршрутам городского транспорта
bus_route_nums = ['68', '103', '153', '27', '92', '168', '114', '9', '127', '172', '171', '134А','13', '13А', '39', '187', '49', '181','71', '66', '35', '2', '6', '71', '26']
troll_route_nums = [37, 38, 27, 35]

bus_df = r_df[r_df['id_transport'] == 1]
tram_df = r_df[r_df['id_transport'] == 2]
troll_df = r_df[r_df['id_transport'] == 3]

print("Результаты поиска данных о нужных автобусах в документе rbs.csv")
print()
for bus_route_num in bus_route_nums:
    print('----------------------------------------')
    print('Маршрут ', bus_route_num)
    print('Количество остановок в прямом направлении: ', bus_df[bus_df['route_number'] == bus_route_num]['number_of_stops_in_direction1'].values[0])
    print('Количество остановок в обратном направлении: ', bus_df[bus_df['route_number'] == bus_route_num]['number_of_stops_in_direction2'].values[0])

print()
print("Результаты поиска данных о нужных троллейбусах в документе rbs.csv")
print()
for troll_route_num in troll_route_nums:
    print('----------------------------------------')
    print('Маршрут ', troll_route_num)
    print('Количество остановок в прямом направлении: ', troll_df[troll_df['route_number'] == troll_route_num]['number_of_stops_in_direction1'].values[0])
    print('Количество остановок в обратном направлении: ', troll_df[troll_df['route_number'] == troll_route_num]['number_of_stops_in_direction2'].values[0])

In [ ]:
stops_in_dir1 = r_df['stops_in_direction1'].values
clusters_column_data1 = []

for stops_list in stops_in_dir1:
    if stops_list == '-1':
        clusters_column_data1.append('-1')
    else:
        stops = stops_list.split(',')
        clusters_str = find_cluster_id(int(stops[0]))
        for stop in stops[1:]:
            clusters_str = clusters_str + ',' + find_cluster_id(stop)
        clusters_column_data1.append(clusters_str)
    
stops_in_dir2 = r_df['stops_in_direction2'].values
clusters_column_data2 = []
for stops_list in stops_in_dir2:
    if stops_list == '-1':
        clusters_column_data2.append('-1')
    else:
        stops = stops_list.split(',')
        clusters_str = find_cluster_id(int(stops[0]))
        for stop in stops[1:]:
            clusters_str = clusters_str + ',' + find_cluster_id(int(stop))
        clusters_column_data2.append(clusters_str)

        
r_df['clusters_in_direction1'] = None
r_df['clusters_in_direction2'] = None
r_df.loc[:,'clusters_in_direction1'] = pd.Series(np.array(clusters_column_data1))
r_df.loc[:,'clusters_in_direction2'] = pd.Series(np.array(clusters_column_data2)) 

r_df.to_csv('resources/social_routes_info.csv', encoding='utf-8-sig')